In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import datetime 
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import sqlalchemy

init_notebook_mode(connected=True)

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

WINDOW = 22

In [3]:
"""
Парсинг данных

"""

def connect(user, password, db, host: str, port: int, echo=False):
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    eng = sqlalchemy.create_engine(url, client_encoding='utf8', echo=echo)
    meta = sqlalchemy.MetaData(bind=eng)
    return eng, meta

def get_data_frame(pair: str = 'USDT_BTC', exchange: str = 'poloniex') -> pd.DataFrame:

    engine, meta = connect(user='postgres', password='password', db='btccandles', host='94.230.125.199', port=16432)
    df = pd.read_sql_query(
        "SELECT date, time, open, close, low, high, volume, pair.\"name\""
        "FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN ("
        "SELECT pair.id FROM pair, exchange WHERE ("
        "SELECT pair.id FROM pair WHERE pair.name = '" + pair +
        "') = pair.alias_id AND pair.exchange_id = (SELECT exchange.id FROM exchange WHERE exchange.\"name\" = '" + exchange + "')) ORDER BY candlestick.date;",
        con=engine)
    return df


"""
Загрузка данных
"""
def load_data(X, seq_len, train_size=1):
    
    amount_of_features = X.shape[1] 
    X_mat = X.as_matrix() 
    
    data = []
    
    for index in range(len(X_mat)  - seq_len):
        data.append(X_mat[index: index + seq_len +1])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))

    return x_train, y_train

"""
Собираем LSTM
"""

def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

def predictNumDay(num, pathModel, data, scaler):
    m = load_model(pathModel)
    
    prediction = []
    lastbatch = np.array(data[-WINDOW:])
    for i in np.arange(num):    
        res = m.predict([lastbatch.reshape(1,22, 5)])
        prediction.append(scaler.inverse_transform(res))
    
    return np.array(prediction).reshape(num)


In [4]:
"""
typeBlockchain:
USDT_BTC
USDT_LTC
USDT_ETH
USDT_ETC
USDT_XRP
"""

def nextDayPrediction(typeBlockchain, stock, N = 1):    
    
    df = get_data_frame(typeBlockchain, stock)

    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    all_df = df.copy()

    x = all_df[['open', 'low', 'high', 'volume']].copy()    
    x = pd.ewma(x, halflife=3)

    y = all_df['close'].copy()
    y = pd.ewma(y, halflife=3)
    
    x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)

    y = y_scaler.fit_transform(y.values.reshape(-1, 1))
    x['close'] = y
    
    X_train, y_train = load_data(x, WINDOW)
    
    #print (X_train.shape, y_train.shape)
    
    model = build_model(input_shape=(WINDOW, 5))
    
    print('START FIT MODEL...')
    
    start = time.time()
    model.fit(X_train, y_train, batch_size=32, epochs=500,
              verbose=1)
    end = time.time()

    print ('Learning time: ', end-start)
    
    today = time.strftime("_%d_%m_%Y")
    
    pathModel = "../models/model_5f_" + typeBlockchain + today +".h5"
    save_model(model, pathModel)
    
    #model = load_model(pathModel)
    
    trainPredict = model.predict(X_train)
    trainPredict = y_scaler.inverse_transform(trainPredict)
    trainY = y_scaler.inverse_transform([y_train])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))
    
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    """
    
    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.legend(loc='best')
    plt.show()
    
    """
    
    lastDate =str(df.date[df.last_valid_index()]).split('-')
    currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])+1)
    predictionDate = pd.date_range(currentData,periods=N)
    predictNday =  (predictNumDay(N, pathModel, x, y_scaler))

    prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)
    
    """Actual = pd.DataFrame(prices, columns=["close"], index=df.date).close
    Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
    pred = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close

    ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
    TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

    iplot([ActualValues,TrainingValues])
    
    our_Predict = go.Scatter( x = prediction.index, y = prediction.predictionPrice, name = 'NextDayValues')

    iplot([our_Predict, ActualValues])"""
    
    return prediction

In [7]:
USDT_BTC = nextDayPrediction('USDT_BTC', 'poloniex' ,N = 1)

/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning:

pd.ewm_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.ewm(halflife=3,min_periods=0,adjust=True,ignore_na=False).mean()

/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning:

pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(halflife=3,min_periods=0,adjust=True,ignore_na=False).mean()



START FIT MODEL...
Epoch 1/500
703/703 [==============================] - 1s - loss: 0.0394 - acc: 0.0014       
Epoch 2/500
703/703 [==============================] - 0s - loss: 0.0040 - acc: 0.0028     
Epoch 3/500
703/703 [==============================] - 0s - loss: 0.0019 - acc: 0.0028     
Epoch 4/500
703/703 [==============================] - 0s - loss: 0.0010 - acc: 0.0028     
Epoch 5/500
703/703 [==============================] - 0s - loss: 0.0013 - acc: 0.0028     
Epoch 6/500
703/703 [==============================] - 0s - loss: 0.0010 - acc: 0.0028           
Epoch 7/500
703/703 [==============================] - 0s - loss: 9.7053e-04 - acc: 0.0028     
Epoch 8/500
703/703 [==============================] - 0s - loss: 0.0010 - acc: 0.0028       
Epoch 9/500
703/703 [==============================] - 0s - loss: 0.0013 - acc: 0.0028     
Epoch 10/500
703/703 [==============================] - 0s - loss: 0.0012 - acc: 0.0028     
Epoch 11/500
703/703 [========================

703/703 [==============================] - 0s - loss: 3.3862e-04 - acc: 0.0028     
Epoch 169/500
703/703 [==============================] - 0s - loss: 1.8583e-04 - acc: 0.0028     
Epoch 170/500
703/703 [==============================] - 0s - loss: 2.7690e-04 - acc: 0.0028     
Epoch 171/500
703/703 [==============================] - 0s - loss: 2.9209e-04 - acc: 0.0028     
Epoch 172/500
703/703 [==============================] - 0s - loss: 2.7545e-04 - acc: 0.0028     
Epoch 173/500
703/703 [==============================] - 0s - loss: 2.7450e-04 - acc: 0.0028        
Epoch 174/500
703/703 [==============================] - 0s - loss: 2.4733e-04 - acc: 0.0028     
Epoch 175/500
703/703 [==============================] - 0s - loss: 2.9699e-04 - acc: 0.0028     
Epoch 176/500
703/703 [==============================] - 0s - loss: 3.4783e-04 - acc: 0.0028     
Epoch 177/500
703/703 [==============================] - 0s - loss: 2.6253e-04 - acc: 0.0028     
Epoch 178/500
703/703 [========

703/703 [==============================] - 0s - loss: 1.8191e-04 - acc: 0.0028     
Epoch 335/500
703/703 [==============================] - 0s - loss: 1.7619e-04 - acc: 0.0028       
Epoch 336/500
703/703 [==============================] - 0s - loss: 1.8088e-04 - acc: 0.0028     
Epoch 337/500
703/703 [==============================] - 0s - loss: 1.5558e-04 - acc: 0.0028     
Epoch 338/500
703/703 [==============================] - 0s - loss: 1.4896e-04 - acc: 0.0028     
Epoch 339/500
703/703 [==============================] - 0s - loss: 1.6998e-04 - acc: 0.0028       
Epoch 340/500
703/703 [==============================] - 0s - loss: 1.7212e-04 - acc: 0.0028     
Epoch 341/500
703/703 [==============================] - 0s - loss: 2.1541e-04 - acc: 0.0028     
Epoch 342/500
703/703 [==============================] - 0s - loss: 2.9754e-04 - acc: 0.0028     
Epoch 343/500
703/703 [==============================] - 0s - loss: 2.8067e-04 - acc: 0.0028     
Epoch 344/500
703/703 [=======

703/703 [==============================] - 0s - loss: 1.7313e-04 - acc: 0.0028     
Learning time:  456.44448041915894
Train Score: 24.85 RMSE


In [8]:
USDT_BTC

,predictionPrice
2017-07-26,2629.826416


In [ ]:
USDT_LTC = nextDayPrediction('USDT_LTC', 'poloniex',N = 1)

2017-07-26 11:53:02,522 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 11:53:02,523 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,524 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 11:53:02,525 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,527 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 11:53:02,529 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,531 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 11:53:02,532 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 11:53:02,533 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_ETH = nextDayPrediction('USDT_ETH', 'poloniex',N = 1)

Learning time:  408.56155252456665
Train Score: 3.41 RMSE


In [ ]:
USDT_ETC = nextDayPrediction('USDT_ETC', 'poloniex',N = 1)

2017-07-26 12:06:48,766 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:06:48,767 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,768 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:06:48,769 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,770 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,771 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:06:48,773 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:06:48,775 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:06:48,776 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [ ]:
USDT_XRP = nextDayPrediction('USDT_XRP', 'poloniex',N = 1)

2017-07-26 12:10:19,342 INFO sqlalchemy.engine.base.Engine select version()
2017-07-26 12:10:19,343 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-26 12:10:19,345 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,348 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-26 12:10:19,350 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,351 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-26 12:10:19,352 INFO sqlalchemy.engine.base.Engine {}
2017-07-26 12:10:19,353 INFO sqlalchemy.engine.base.Engine SELECT date, time, open, close, low, high, volume, pair."name"FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN (SELEC

In [86]:
print ('predictionValues on', datetime.date.today())

allCurrency = ['USDT_BTC', 'USDT_LTC', 'USDT_ETH', 'USDT_ETC', 'USDT_XRP']
predictedValues = [USDT_BTC, USDT_LTC, USDT_ETH, USDT_ETC, USDT_XRP]

for index in np.arange(len(allCurrency)):
    print(allCurrency[index], '\n' , predictedValues[index] )


predictionValues on 2017-07-26
USDT_BTC 
             predictionPrice
2017-07-26      2545.130371
USDT_LTC 
             predictionPrice
2017-07-26        42.890644
USDT_ETH 
             predictionPrice
2017-07-26       193.827957
USDT_ETC 
             predictionPrice
2017-07-26        14.239629
USDT_XRP 
             predictionPrice
2017-07-26         0.177823


In [63]:
def checkTrend(typeBlockchain, stock, predicted):
    
    df = get_data_frame(typeBlockchain, stock)
    lastValue = df.close[df.shape[0]-1]
    change = (predicted.values * 100 / lastValue) - 100
    if change > 0:
        mark = 'PositiveTrend'
    else: mark = 'NegativeTrend'
        
    print ('-'*10 + '\nSTOCK:%s, type:%s\n'%(stock, typeBlockchain))
    print ('Change between last (%f) and predicted values(%f) = %f perc.\n%s' \
           % (lastValue, predicted.values, change, mark))

In [84]:
stock = 'poloniex'

for index in np.arange(len(allCurrency)):
    checkTrend(allCurrency[index], stock , predictedValues[index] )

----------
STOCK:poloniex, type:USDT_BTC

Change between last (2565.999999) and predicted values(2545.130371) = -0.813316 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_LTC

Change between last (42.110000) and predicted values(42.890644) = 1.853821 perc.
PositiveTrend
----------
STOCK:poloniex, type:USDT_ETH

Change between last (203.000000) and predicted values(193.827957) = -4.518250 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_ETC

Change between last (14.620000) and predicted values(14.239629) = -2.601715 perc.
NegativeTrend
----------
STOCK:poloniex, type:USDT_XRP

Change between last (0.174540) and predicted values(0.177823) = 1.881226 perc.
PositiveTrend
